# 작업형1

## 문제1
주어진 데이터는 학생들의 성적 데이터이다.

test preparation course 칼럼의 값 이 none인 데이터는 결측값으로 간주하여 제거한 뒤, 

학생들이 가장 많은 인종 (race/ethnicity) 그룹을 추출하여 새로운 데이터프레임을 만든다. 

새로 생성된 데이터프레임의 math score 칼럼을 표준화하고, 

표준화된 math score 칼럼의 최댓값을 출력하시오. (단, 연산 결과는 반올림 후 소수점 3자리까지 출력한다.)

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')  # 경고 메시지 무시
df = pd.read_csv('/kaggle/input/bigdata-csvfile/student_performance_data.csv')

# test preparation course 결측값 제거
df = df[df['test preparation course'] != 'none']

# 학생들이 가장 많은 인종 (race/ethnicity) 그룹을 추출하여 새로운 데이터프레임
student_group = df['race/ethnicity'].value_counts() 
#print(student_group) #group C 117
new_data = df[df['race/ethnicity'] == 'group C']

# math score컬럼 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
new_data['scaler'] = scaler.fit_transform(new_data[['math score']])
print(round(new_data['scaler'].max(),3)) # 2.145

2.145


## 문제2

주어진 데이터는 2000년 1월부터 2023년 12월까지 주식, 외환, 주요 원자재에 대한 가격 데이터이다. 

원 달러 환율(dollar exchange rate)과 가장 높은 상관계수를 갖는 칼럼의 평균값을 구하시오. 

(단, 출력값은 반올림하여 소수점 3자리까 지 출력한다.)

- date : 날짜
- BAC : 뱅크오브아메리카 주식 가격
- dollar exchange rate : 원 달러 환율
- KOSPI : 코스피 지수
- gold price : 금 거래가격
- silver price : 은 거래가격
- natural gas price : 천연가스 거래가격


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/Financial_Data.csv')

# 원 달러 환율(dollar exchange rate)과 가장 높은 상관계수를 갖는 칼럼의 평균값
corr = df.drop(columns = 'date').corr()
corr = (corr.drop('dollar exchange rate'))
result_corr = np.abs(corr['dollar exchange rate']) # 가장 높은 상관계수를 갖는 칼럼 : natural gas price, 0.366707
print(round(df['natural gas price'].mean(),3)) # 평균값 : 4.514

4.514


## 문제3

주어진 공기질 데이터에서 결측값을 제거하고, Wind 컬럼의 이상치 개수를 구하시오.

(단, 이상치 범위는 Q1-1.5*IQR 미만 또는 Q3+1.5*IQR 초과로 하고,

IQR = Q3 - Q1와 같이 계산한다.)

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/airquality.csv')
# 결측값 제거
df = df.dropna()

# 이상값
Q3 = df['Wind'].quantile(0.75)
Q1 = df['Wind'].quantile(0.25)
IQR = Q3-Q1

outlier = (df['Wind'] < (Q1 - (IQR * 1.5))) | (df['Wind'] > (Q3 + (IQR * 1.5)))
print(len(df[outlier])) # 이상치 : 3개

3


# 작업형2

1. 전체 데이터의 80%인 1,074행을 train 데이터로 사용하고, 나머지를 test 데이터로 사용할 수 있도록 데이터 슬라이싱 작업을 한 후에 데이터를 분석하도록 한다.
2. train 데이터를 활용하여 예측 모델을 모델링하고, test 데이터에 적용하여 목표변 수인 mosquito_indicator를 예측하고, 예측 결과를 제출한다. (단, 제출 결과는 test 데이터의 개수인 268행이 될 수 있도록 한다.)
3. 제출 데이터 칼럼은 date와 mosquito_Indicator 두 개만 존재해야 하고, 평가지표 는 RMSB로 한다.
4. 예측 결과 파일명은 7th_test_type2.csv로 한다.

- date : 날짜
- mosquito_Indicator : 모기수
- rain(mm) : 강수량
- mean_T : 일 평균 온도
- min_T : 일 최소 온도
- max_T : 일 최대 온도

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/mosquito_Indicator.csv')

train = df[:1074]
test = df[1074:]
y = train['mosquito_Indicator']
train = train.drop(columns = ['mosquito_Indicator', 'date'])
test = test.drop(columns = ['mosquito_Indicator', 'date'])
# 데이터분할
from sklearn.model_selection import train_test_split
X = train
y = y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)

# 모델링 학습
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 모델 성능 평가
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, pred)
print('mse', mse)
rmse = np.sqrt(mse)
print(rmse)

# 최종결과 예측
model.fit(train, y)
pred2 = model.predict(test)

# 데이터프레임 생성
result = pd.DataFrame({'pred':pred2})
result.to_csv('result.csv', index=False)
result_read = pd.read_csv('result.csv')
print(result_read)

mse 34107.155662744204
184.68122715301683
       pred
0    10.502
1    43.196
2     9.653
3    40.964
4    22.275
..      ...
263   3.100
264   5.190
265   4.120
266   4.261
267   4.114

[268 rows x 1 columns]


# 작업형3

## 문제1

1. train 데이터의 전체 무게(Whole_weight) 칼럼을 사용하여 전복의 성별(Sex)을 예 측하는 로지스틱 회귀 분석을 했을 때, 전체 무게(Whole_weight)가 1g(단위) 증가 할 때 수컷(1)의 오즈비를 구하시오.
2. train 데이터의 전복 성별(Sex)을 종속변수로 하고, Length, Diameter, Height, Rings를 독립변수로 하여 로지스틱 회귀 분석을 했을 때, 잔차 이탈도(deviance)를 구하시오.
3. train 데이터를 활용하여 전복의 성별(Sex)을 분류하는 로지스틱 회귀 모델을 만들 었을 때, 이 모델을 활용하여 test 데이터에 적용한 결과에 대한 오분류율을 구하시오.
  
(단, 원본 데이터의 성별(Sex)의 값이 2인 데이터를 제외하고, train 데이터(상위 2,500)와 test 데이터(하위 395행)로 슬라이싱하여 사용한다. 로지스틱 회귀 모형에 서 train_test_split 함수를 활용하여 데이터 분할 시, random state는 100으로 하고, test size는 20%로 한다.) (모든 결팟값은 반올림하여 소수점 3자리까지 출력한다.)

In [5]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
from sklearn.model_selection import train_test_split

# 1. 오즈비
df = pd.read_csv('/kaggle/input/bigdata-csvfile/abalone_data.csv')
cond = df['Sex'] !=2
df = df[cond]

print(df.shape)

train = df[:2500]
test = df[2500:]
formula = ('Sex ~ Whole_weight')
model = logit(formula, data= train).fit()
odds_ratio = np.exp(model.params)
print(odds_ratio)

# 2. 
formula = ('Sex~  Length + Diameter + Height + Rings')
model = logit(formula, data=train).fit()
print(round(-2 * model.llf,3)) #3424.691

# 3.
X = train.drop(columns = 'Sex')
y = train['Sex']

X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=100, test_size=0.2)
train_df = X_train.copy()
train_df['Sex'] = y_train

test_df = X_test.copy()
test_df['Sex'] = y_test

formula = ('Sex~  Length + Diameter + Height + Rings')
model = logit(formula, data=train_df).fit()
pred = model.predict(test_df)
class_pred = (pred > 0.5).astype(int)
accuracy = (class_pred == test_df['Sex']).mean()
error_rate = 1 - accuracy
print(round(error_rate,3)) 

(2835, 9)
Optimization terminated successfully.
         Current function value: 0.688890
         Iterations 4
Intercept       0.671486
Whole_weight    1.284140
dtype: float64
Optimization terminated successfully.
         Current function value: 0.684938
         Iterations 5
3424.691
Optimization terminated successfully.
         Current function value: 0.684483
         Iterations 5
0.482


## 문제2
1. latest_price와 가장 큰 상관계수를 갖는 변수의 상관계수를 구하시오.
2. ram_gb, ssd, hdd, os, graphic_card_gb 칼럼을 독립변수로 하는 다중 선형 회귀 모형으로 latest_price 칼럼을 예측할 때, 결정계수(R-squared)를 구하시오.
3. 2번 문제의 다중 선형 회귀 모형에서 가장 큰 값을 갖는 칼럼의 T-검정 통계량을 구 하시오. (단, 데이터 분석에는 ram_gb, ssd, hdd, os, graphic_card_gb, latest_ price, old_price, discount, ratings, reviews 칼럼만을 사용한다.)

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/new_laptop_data.csv')

# 1.
data = df[['ram_gb', 'ssd', 'hdd', 'warranty', 'latest_price', 
           'old_price', 'discount', 'star_rating', 'ratings', 'reviews']]
data_corr = data.corr().drop('latest_price')
abs_corr = np.abs(data_corr['latest_price'])
print(round(abs_corr.max(),3)) # 0.748

# 2.
from statsmodels.formula.api import ols
formula = ('latest_price ~ ram_gb + ssd + hdd + os + graphic_card_gb')
model = ols(formula, data = df).fit()
print(model.summary()) # 0.617

# 3.
print(f't 검정통계량 : t값 {17.865}')

0.748
                            OLS Regression Results                            
Dep. Variable:           latest_price   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     238.8
Date:                Wed, 11 Jun 2025   Prob (F-statistic):          1.76e-181
Time:                        19:04:48   Log-Likelihood:                -10472.
No. Observations:                 896   AIC:                         2.096e+04
Df Residuals:                     889   BIC:                         2.099e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         8.38e+04   5433.95